# Bos taurus mitokondriumok kódoló mutációi

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import re
from math import floor

In [2]:
infile=open('bos_sequence.fasta','r' )
adat=infile.read()
sorszerint=adat.split("\n")

Teljes adatokat az egyes mitokondrium szekvenciák szerint tördeljük. (Címsorok már nem lesznek benne a tömbbben.)

In [3]:
AllMitoSeq=[]
seq=""
elsosore=True
for sor in sorszerint:
    if len(sor)>0:
        if sor[0]=='>':
            if elsosore!=True:
                AllMitoSeq.append(seq)
            else:
                elsosore=False
            seq=""
        else:
            seq+=sor

Leghosszabb és legrövidebb szekvenciák hosszának meghatározása.

In [4]:
LenSeq=[]
for seq in AllMitoSeq:
    LenSeq.append(len(seq))
    
print(min(LenSeq),max(LenSeq))

16337 16448


A minimum és maximum közé eső (és megegyező) hosszúságú szekvenciákból hány darab van.

In [5]:
LenStat=[]

for i in np.arange(min(LenSeq),max(LenSeq)+1):
    if LenSeq.count(i)>0:
        LenStat.append([i,LenSeq.count(i)])

print(LenStat)

[[16337, 10], [16338, 26], [16339, 87], [16340, 115], [16341, 12], [16342, 43], [16344, 1], [16356, 1], [16383, 1], [16426, 1], [16448, 1]]


A leggyakoribb hosszúságú szekvenciák kiválogatása.

In [6]:
maxcount=0

for i in range(len(LenStat)):
    if LenStat[i][1]>maxcount:
        maxcount=LenStat[i][1]
        maxindex=i
    
print(LenStat[maxindex])

[16340, 115]


Mégis a 16338 hosszúakat használtam, mert a [referencia genom](http://www.ncbi.nlm.nih.gov/nuccore/NC_006853.1) is ilyen hoszzú.

In [7]:
Mito=[]

for sor in AllMitoSeq:
    if len(sor)==(LenStat[1][0]):
        Mito.append(sor)

Különbségi mátrix: Minden szekvenciát minden másik szekvenciával összehasonlítom

In [1]:
def compare(x1,x2):
    k=0

    for i in range(len(x1)):
        if x1[i]!=x2[i]:
            k=k+1
    return k

In [9]:
matrixom=np.zeros((LenStat[maxindex][1],LenStat[maxindex][1]))

for i in range(len(Mito)):
    for j in range(len(Mito)):
        matrixom[i,j]=compare(Mito[i],Mito[j])
        

A ftp://ftp.ncbi.nlm.nih.gov/genomes/Bos_taurus/CHR_MT/bt_ref_Bos_taurus_UMD_3.1.1_chrMT.gbk.gz fileból kiszedem a géneket kódoló régiók kezdeti és végső nukleotidjának pozícióit ('gene'-el kezdődő sorok)

In [11]:
reffile=open('bt_ref_Bos_taurus_UMD_3.1.1_chrMT.gbk', 'r')
adat2=reffile.read()
byline=adat2.split("\n")

In [12]:
GeneSeq=[]

for i in range(len(byline)):
    if byline[i][0:9]=='     gene':
        kulon=byline[i].split()
        if kulon[1][0]!='c':
            #A complementtel kezdődőt kidobtam
            GeneSeq.append(kulon[1])
    

In [14]:
Start=[]
End=[]

for pos in GeneSeq:
    startend=pos.split('..')
    Start.append(int(startend[0]))
    End.append(int(startend[1]))
    
print(Start,End)

[3101, 4266, 5687, 7374, 8129, 8290, 8970, 9823, 10239, 10529, 12109, 14514] [4056, 5307, 7231, 8057, 8329, 8970, 9750, 10168, 10535, 11906, 13929, 15653]


Mito-ban az adott, referencia genommal egyező hosszúságú mktokondrium szekvenciák vannak (26)  
MitoGene-ben ezek vannak gének szerint szétválogatva  
- Első index:egyedek
- Második index:gének

In [15]:
MitoGene=[]
for j in range(len(Mito)):
    Individ=[]
    for i in range(len(Start)):
        Individ.append(Mito[j][Start[i]-1:End[i]-1])
        if i == len(Start)-1:
            MitoGene.append(Individ)

A referencia genomból is kiválogatom a géneket.

In [17]:
infile=open('bos_ref_sequence.fasta','r' )
adat=infile.read()
sorszerint=adat.split("\n")

In [18]:
RefSeq=""

for sor in sorszerint:
    if len(sor)>0:
        if sor[0]!='>':
            RefSeq+=sor

In [20]:
RefMitoGene=[]
for i in range(len(Start)):
    RefMitoGene.append(RefSeq[Start[i]-1:End[i]-1])

És ezeket összehasonlítom az összes mitokondrium génszekvenciával.

In [93]:
szekvmatrix=np.zeros((len(RefMitoGene),len(MitoGene)))

for i in range(len(RefMitoGene)):
    for j in range(len(MitoGene)):
        szekvmatrix[i,j]=compare(RefMitoGene[i],MitoGene[j][i])

Elejétől indulva aminosavakká átkonvertálom a szekvenciákat. és ezeket is összehasonlítom.  
Nem minden gén osztható 3-al, de az egyszerűség kedvéért az elejétől indulva addig veszem, amíg van 3 bázis egymás után

In [25]:
map = {"TTT":"Phe", "TTC":"Phe", "TTA":"Leu", "TTG":"Leu",
    "TCT":"Ser", "TCC":"Ser", "TCA":"Ser", "TCG":"Ser",
    "TAT":"Tyr", "TAC":"Tyr", "TAA":"STOP", "TAG":"STOP",
    "TGT":"Cys", "TGC":"Cys", "TGA":"STOP", "TGG":"Trp",
    "CTT":"Leu", "CTC":"Leu", "CTA":"Leu", "CTG":"Leu",
    "CCT":"Pro", "CCC":"Pro", "CCA":"Pro", "CCG":"Pro",
    "CAT":"His", "CAC":"His", "CAA":"Gln", "CAG":"Gln",
    "CGT":"Arg", "CGC":"Arg", "CGA":"Arg", "CGG":"Arg",
    "ATT":"Ile", "ATC":"Ile", "ATA":"Ile", "ATG":"Met",
    "ACT":"Thr", "ACC":"Thr", "ACA":"Thr", "ACG":"Thr",
    "AAT":"Asn", "AAC":"Asn", "AAA":"Lys", "AAG":"Lys",
    "AGT":"Ser", "AGC":"Ser", "AGA":"Arg", "AGG":"Arg",
    "GTT":"Val", "GTC":"Val", "GTA":"Val", "GTG":"Val",
    "GCT":"Ala", "GCC":"Ala", "GCA":"Ala", "GCG":"Ala",
    "GAT":"Asp", "GAC":"Asp", "GAA":"Glu", "GAG":"Glu",
    "GGT":"Gly", "GGC":"Gly", "GGA":"Gly", "GGG":"Gly",}

In [26]:
RefAmino=[]

for i in range(len(RefMitoGene)):
    ByGene=[]
    for j in range(0,int(floor((len(RefMitoGene[i])/3))*3),3):
        kodon="".join(RefMitoGene[i][j:j+3])
        ByGene.append(map[kodon])
        if j==(int(floor((len(RefMitoGene[i])/3))*3))-3:
            RefAmino.append(ByGene)
        

In [73]:
AllAmino=[]

for i in range(len(MitoGene)):
    ByIndivid=[]
    for x in range(len(MitoGene[i])):
        ByGene=[]
        for j in range(0,int((floor(len(MitoGene[i][x])/3)))*3,3):
            kodon="".join(MitoGene[i][x][j:j+3])
            ByGene.append(map[kodon])
            if j==(int(floor((len(MitoGene[i][x])/3))*3))-3:
                ByIndivid.append(ByGene)
                if x==len(MitoGene[i])-1:
                    AllAmino.append(ByIndivid)
            
        

Összehasonlítom a referencia aminosav génszekvenciákkal a minta aminosav génszekvanciákkal.

In [88]:
aminomatrix=np.zeros((len(RefAmino),len(AllAmino)))


for i in range(len(RefAmino)):
    for j in range(len(AllAmino)):
        aminomatrix[i,j]=compare(RefAmino[i],AllAmino[j][i])
        

A referencia szekvencia génjeit összehasonlítom a mintagénekkel, és ha a tripletek által kódolt aminosavak különböznek, kiírom a tripletet és annak helyét.  
Az ujb-ben vannak az aminosav szekvenciában is jelen levő mutációk helyei.

In [84]:
AllDiff = []
for i in range(len(MitoGene)):
    for g in range(len(RefMitoGene)):
        lista=[]
        for b in range(0,int(floor((len(RefMitoGene[g])/3))*3),3):
            RefCodon="".join(RefMitoGene[g][b:b+3])
            AllCodon="".join(MitoGene[i][g][b:b+3])
            RefAA = map[RefCodon]
            AllAA = map[AllCodon]
            if RefAA!=AllAA:
                for nn in range(3):
                    if RefCodon[nn]!=AllCodon[nn]:
                        ujb=b+nn
                        AllDiff.append([i,g,b,ujb,RefCodon,AllCodon,RefAA,AllAA])
                

In [91]:
infile=open('muthelyek.csv',"w")
infile.write("Egyed,Gen,Tripletkezdet,Mutans bazishely,Referencia triplet,Minta triplet,Referencia aminosav,Minta aminosav \n")
for i in AllDiff:
    infile.write(str(i).replace('[','').replace(']','')+'\n')
infile.close()